Problem statement

For the March edition of the 2022 Tabular Playground Series you're challenged to forecast twelve-hours of traffic flow in a U.S. metropolis. The time series in this dataset are labelled with both location coordinates and a direction of travel -- a combination of features that will test your skill at spatio-temporal forecasting within a highly dynamic traffic network.

Import libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

Load files

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read files

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/test.csv")
submission = pd.read_csv("/kaggle/input/tabular-playground-series-mar-2022/sample_submission.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
submission

Analyse target

In [ ]:
sns.displot(train['congestion'])

In [ ]:
sns.violinplot(train['congestion'])

In [ ]:
train.isnull().sum()

Drop target

In [ ]:
target = train["congestion"]

train.drop(["congestion"], axis=1, inplace=True)
train

Combine train and test

In [ ]:
combi = train.append(test)
combi

Timestamp data

In [ ]:
combi['time'] = pd.to_datetime(combi['time'], format="%Y-%m-%d %H:%M:%S")
combi

Month

In [ ]:
combi['month'] = pd.DatetimeIndex(combi['time']).month
combi

Day of Week

In [ ]:
combi["weekday"] = combi['time'].dt.dayofweek
combi

In [ ]:
sns.displot(combi['weekday'])

Hour

In [ ]:
combi['hour'] = pd.to_datetime(combi['time'], format='%Y-%m-%d %H:%M:%S').dt.hour
combi

Analyse direction

In [ ]:
sns.displot(combi['direction'])

Replace direction with numbers

In [ ]:
combi['direction'].replace({'EB':1, 'NB':2, 'SB':3, 'WB':4, 'NE':5, 'SW':6, 'NW':7,'SE':8}, inplace=True)
combi

Features

In [ ]:
features = ['x', 'y', 'direction', 'weekday', 'month', 'hour']
combi[features]

Define X and y

In [ ]:
y = target
X = combi[features][: len(train)]
X_test = combi[features][len(train) :]

Split into training and validating

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10, random_state=42, stratify=y, shuffle=True)
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape

Select model

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42).fit(X, y)
print(model.score(X, y))

Predict on validation set

In [ ]:
#y_pred = model.predict(X_val)
#print(model.score(X_val, y_val))

In [ ]:
#from sklearn.metrics import mean_squared_error

#rmse = mean_squared_error(y_val, y_pred, squared=False)
#rmse

In [ ]:
#df=pd.DataFrame({'Actual': y_val, 'Predicted':y_pred})
#df

In [ ]:
#fig, ax = plt.subplots()
#ax.scatter(y_val, y_pred, edgecolors=(0, 0, 0))
#ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
#ax.set_xlabel('Measured')
#ax.set_ylabel('Predicted')
#plt.show()

Predict on test set

In [ ]:
preds = model.predict(X_test)
preds = preds.astype(int)
preds[preds < 0] = 0
preds

Prepare submission

In [ ]:
submission.congestion = preds
submission.to_csv('submission.csv', index=False)
submission = pd.read_csv("submission.csv")
submission